In [31]:
!pip install pdfplumber
!pip install tiktoken
!pip install openai==0.28.1
!pip install chromaDB
!pip install sentence-transformers
!pip install PyPDF2


In [78]:
!pip install openai==0.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.13.3
    Uninstalling openai-1.13.3:
      Successfully uninstalled openai-1.13.3


In [1]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import pdfplumber
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import openai
import chromadb
import re

In [2]:
# Open the PDF file
with pdfplumber.open("/content/Principal-Sample-Life-Insurance-Policy.pdf") as pdf:

    # Get one of the pages from the PDF and examine it
    single_page = pdf.pages[6]

    # Extract text from the first page
    text = single_page.extract_text()

    # Extract tables from the first page
    tables = single_page.extract_tables()

    # Print the extracted text
    print(text)

Section A – Eligibility
Member Life Insurance Article 1
Member Accidental Death and Dismemberment Insurance Article 2
Dependent Life Insurance Article 3
Section B - Effective Dates
Member Life Insurance Article 1
Member Accidental Death and Dismemberment Insurance Article 2
Dependent Life Insurance Article 3
Section C - Individual Terminations
Member Life Insurance Article 1
Member Accidental Death and Dismemberment Insurance Article 2
Dependent Life Insurance Article 3
Termination for Fraud Article 4
Coverage While Outside of the United States Article 5
Section D - Continuation
Member Life Insurance Article 1
Dependent Insurance - Developmentally Disabled or
Physically Handicapped Children Article 2
Section E - Reinstatement
Reinstatement Article 1
Federal Required Family and Medical Leave Act (FMLA) Article 2
Reinstatement of Coverage for a Member or Dependent When
Coverage Ends due to Living Outside of the United States Article 3
Section F - Individual Purchase Rights
Member Life In

In [3]:
pdf_path='/content/Principal-Sample-Life-Insurance-Policy.pdf'

In [4]:
# extracted_text=extract_text_from_pdf(pdf_path)

In [5]:
def extract_pdf_sections_to_df(pdf_file):
    sections = []
    current_section = None
    section_number = 0

    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            lines = text.split('\n')

            for line in lines:
                if re.match(r'^PART [IVXLCDM]+ - .+$', line):
                    if current_section:
                        sections.append(current_section)
                    section_number += 1
                    current_section = {"section_no": section_number, "section_name": line, "content": []}
                elif current_section:
                    current_section["content"].append(line)

    if current_section:
        sections.append(current_section)

    # Create a DataFrame
    df = pd.DataFrame(sections)

    # Post-process the section content for "PART IV - BENEFITS"
    for i, section in df.iterrows():
        if "PART IV - BENEFITS" in section['section_name']:
            content = "\n".join(section['content'])
            subsections = re.split(r'(,?This policy has been updated effective [^,]+,)', content)
            section_name = "PART IV - BENEFITS"
            for j, subsection in enumerate(subsections):
                if subsection.strip():
                    df = df.append({"section_no": len(df) + j + 1, "section_name": section_name, "content": subsection}, ignore_index=True)
            df.drop(index=i, inplace=True)

    return df


pdf_path = '/content/Principal-Sample-Life-Insurance-Policy.pdf'
df = extract_pdf_sections_to_df(pdf_path)




<ipython-input-5-9062257b3398>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"section_no": len(df) + j + 1, "section_name": section_name, "content": subsection}, ignore_index=True)
<ipython-input-5-9062257b3398>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"section_no": len(df) + j + 1, "section_name": section_name, "content": subsection}, ignore_index=True)
<ipython-input-5-9062257b3398>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"section_no": len(df) + j + 1, "section_name": section_name, "content": subsection}, ignore_index=True)
<ipython-input-5-9062257b3398>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a futur

In [6]:
df.head()

,section_no,section_name,content
0,1,PART I - DEFINITIONS,[]
1,2,PART II - POLICY ADMINISTRATION,"[Section A – Contract, Entire Contract Article..."
2,3,PART III - INDIVIDUAL REQUIREMENTS AND RIGHTS,[This policy has been updated effective Januar...
3,5,PART II - POLICY ADMINISTRATION,"[Section A - Contract, Article 1 - Entire Cont..."
4,6,PART II - POLICY ADMINISTRATION,"[GC 6003 Section A - Contract, Page 1, a. be a..."


In [7]:
df['Document Name']=pdf_path.split('/')[2]

In [8]:
# Store the metadata for each page in a separate column

df['Metadata'] = df.apply(lambda x: {'Policy_Name': x['Document Name'][:-4], 'Section_No.': x['section_no']}, axis=1)

In [9]:
df['Metadata'][1]

{'Policy_Name': 'Principal-Sample-Life-Insurance-Policy', 'Section_No.': 2}

In [10]:
with open("/content/openapi_key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

In [11]:
# Import the OpenAI Embedding Function into chroma

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [12]:
# Define the path where chroma collections will be stored

chroma_data_path = '/content/'

In [13]:
# Call PersistentClient()

client = chromadb.PersistentClient(path=chroma_data_path)

In [14]:
# Set up the embedding function using the OpenAI embedding model

model = "text-embedding-ada-002"
embedding_function = OpenAIEmbeddingFunction(api_key=openai.api_key, model_name=model)

In [15]:
# Initialise a collection in chroma and pass the embedding_function to it so that it used OpenAI embeddings to embed the documents

policy_collection = client.get_or_create_collection(name='RAG_on_Policy', embedding_function=embedding_function)

In [16]:
# Convert the page text and metadata from your dataframe to lists to be able to pass it to chroma

documents_list = df["content"].tolist()
metadata_list = df['Metadata'].tolist()

In [17]:
# Ensure all elements in documents_list are text strings
documents_list = [str(document) for document in documents_list]

# Add the text content of the documents to the collection
policy_collection.add(
    documents=documents_list,
    ids=[str(i) for i in range(len(documents_list))],  # Use integer IDs
    metadatas=metadata_list
)


In [98]:
# Add the documents and metadata to the collection alongwith generic integer IDs. You can also feed the metadata information as IDs by combining the policy name and page no.

# policy_collection.add(
#     documents= documents_list,
#     ids = [str(i) for i in range(0, len(documents_list))],
#     metadatas = metadata_list
# )

In [18]:
results_df

NameError: name 'results_df' is not defined

In [19]:
policy_collection.get(
    ids = ['0','1','2'],
    include = ['embeddings', 'documents', 'metadatas']
)

{'ids': ['0', '1', '2'],
 'embeddings': [[-0.020228832960128784,
   -0.018979446962475777,
   -0.005636278074234724,
   -0.016199912875890732,
   -0.031810227781534195,
   0.028525322675704956,
   -0.02664422243833542,
   -0.04085073247551918,
   -0.025759825482964516,
   -0.04180531948804855,
   0.015343591570854187,
   0.012255219742655754,
   -0.01248684711754322,
   -0.019709425047039986,
   -0.005144946277141571,
   0.010444310493767262,
   0.029002616181969643,
   -0.007791119161993265,
   0.02497369423508644,
   -0.01229031477123499,
   -0.017112385481595993,
   0.017112385481595993,
   -0.0066224513575434685,
   -0.02094477415084839,
   -0.0187127236276865,
   0.005134417675435543,
   0.008570230565965176,
   -0.02987297624349594,
   -0.010353063233196735,
   -0.015217248350381851,
   0.015118982642889023,
   -0.0014485515421256423,
   -0.006615431979298592,
   -0.02500177174806595,
   -0.020720165222883224,
   -0.007938518188893795,
   -0.008500040508806705,
   -0.005092303268

In [20]:
cache_collection = client.get_or_create_collection(name='Policy_Cache', embedding_function=embedding_function)

In [21]:
cache_collection.peek()

{'ids': ['What is the date for premium payment and any grace period?'],
 'embeddings': [[-0.013038425706326962,
   -0.013150371611118317,
   0.014065694995224476,
   -0.0449628122150898,
   -0.014013014733791351,
   -0.0010091610020026565,
   -0.030660055577754974,
   0.01613340526819229,
   -0.034453053027391434,
   -0.013196466490626335,
   0.024061821401119232,
   0.02823675237596035,
   -0.03332041949033737,
   0.023311123251914978,
   0.02063758857548237,
   -0.012913309037685394,
   0.008270839229226112,
   -0.01174116786569357,
   0.033452123403549194,
   -0.0005317438626661897,
   0.005218662787228823,
   0.00811938289552927,
   -0.028500154614448547,
   0.021111713722348213,
   0.018122093752026558,
   -0.01479005254805088,
   0.0011655562557280064,
   2.634025258885231e-05,
   0.022362874820828438,
   -0.0023640377912670374,
   -0.0008202519384212792,
   -0.006097768433392048,
   -0.024535946547985077,
   -0.022362874820828438,
   -0.003239851212128997,
   -0.0004531346785370

In [22]:
# Read the user query

query = input()

  When is the premium due? any grace period?


In [23]:
# Searh the Cache collection first
# Query the collection against the user query and return the top 20 results

cache_results = cache_collection.query(
    query_texts=query,
    n_results=1
)

In [24]:
# Implementing Cache in Semantic Search
# Set a threshold for cache search
threshold = 0.2
ids = []
documents = []
distances = []
metadatas = []
results_df = pd.DataFrame()

# If the distance is greater than the threshold, then return the results from the main collection.
if cache_results['distances'][0] == [] or cache_results['distances'][0][0] > threshold:
     # Query the collection against the user query and return the top 10 results
     results = policy_collection.query(
     query_texts=query,
     n_results=10
     )
     # Store the query in cache_collection as document w.r.t to ChromaDB so that it can be embedded and searched against later
     # Store retrieved text, ids, distances and metadatas in cache_collection as metadatas, so that they can be fetched easily if a query indeed matches to a query in cache
     Keys = []
     Values = []
     for key, val in results.items():
       if val is None:
         continue
       for i in range(10):
         Keys.append(str(key)+str(i))
         Values.append(str(val[0][i]))

     cache_collection.add(
         documents= [query],
         ids = [query],  # Or if you want to assign integers as IDs 0,1,2,.., then you can use "len(cache_results['documents'])" as will return the no. of queries currently in the cache and assign the next digit to the new query."
         metadatas = dict(zip(Keys, Values))
     )
     print("Not found in cache. Found in main collection.")
     result_dict = {'Metadatas': results['metadatas'][0], 'Documents': results['documents'][0], 'Distances': results['distances'][0], "IDs":results["ids"][0]}
     results_df = pd.DataFrame.from_dict(result_dict)
     results_df

# If the distance is, however, less than the threshold, you can return the results from cache
elif cache_results['distances'][0][0] <= threshold:
     cache_result_dict = cache_results['metadatas'][0][0]
     # Loop through each inner list and then through the dictionary
     for key, value in cache_result_dict.items():
         if 'ids' in key:
             ids.append(value)
         elif 'documents' in key:
             documents.append(value)
         elif 'distances' in key:
             distances.append(value)
         elif 'metadatas' in key:
             metadatas.append(value)
     print("Found in cache!")
     # Create a DataFrame
     results_df = pd.DataFrame({
       'IDs': ids,
       'Documents': documents,
       'Distances': distances,
       'Metadatas': metadatas
     })

Found in cache!


In [25]:
results_df

,IDs,Documents,Distances,Metadatas
0,7,"['GC 6003 Section A - Contract, Page 4', 'Sect...",0.26278409361839294,{'Policy_Name': 'Principal-Sample-Life-Insuran...
1,10,"['GC 6004 Section B - Premiums, Page 3', 'Sect...",0.32783496379852295,{'Policy_Name': 'Principal-Sample-Life-Insuran...
2,8,"['GC 6004 Section B - Premiums, Page 1', 'b . ...",0.3642001152038574,{'Policy_Name': 'Principal-Sample-Life-Insuran...
3,11,"['GC 6005 Section C - Policy Termination, Page...",0.38574838638305664,{'Policy_Name': 'Principal-Sample-Life-Insuran...
4,9,"['GC 6004 Section B - Premiums, Page 2', 'The ...",0.4074781835079193,{'Policy_Name': 'Principal-Sample-Life-Insuran...
5,23,"['GC 6007 Section B - Effective Dates, Page 7'...",0.4128356873989105,{'Policy_Name': 'Principal-Sample-Life-Insuran...
6,12,"['GC 6005 Section C - Policy Termination, Page...",0.41730251908302307,{'Policy_Name': 'Principal-Sample-Life-Insuran...
7,18,"['GC 6007 Section B - Effective Dates, Page 2'...",0.42553043365478516,{'Policy_Name': 'Principal-Sample-Life-Insuran...
8,71,"GC 6013 Section A - Member Life Insurance, Pag...",0.4263361692428589,{'Policy_Name': 'Principal-Sample-Life-Insuran...
9,32,['GC 6011 Section F - Individual Purchase Righ...,0.4362415075302124,{'Policy_Name': 'Principal-Sample-Life-Insuran...


In [26]:
# Import the CrossEncoder library from sentence_transformers

from sentence_transformers import CrossEncoder, util

In [27]:
# Initialise the cross encoder model

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [28]:
# Test the cross encoder model

scores = cross_encoder.predict([['What does "Date of Issue" refer to?', 'For the purposes of this Group Policy, the term "spouse" will include Civil Union,Partner, except as otherwise provided in this Group Policy.,Date of Issue,The date this Group Policy is placed in force: November 1, 2007'],
                                ['What does "Date of Issue" refer to?', "Dependent,a. A Member's spouse, if that spouse:,(1) is legally married to the Member;"]])

In [29]:
scores

array([  0.37614805, -10.741692  ], dtype=float32)

In [30]:
# Input (query, response) pairs for each of the top 20 responses received from the semantic search to the cross encoder
# Generate the cross_encoder scores for these pairs

cross_inputs = [[query, response] for response in results_df['Documents']]
cross_rerank_scores = cross_encoder.predict(cross_inputs)

In [31]:
cross_rerank_scores

array([  3.8873198,   2.8112326,  -4.489765 ,  -4.911915 ,  -9.4965725,
        -9.795955 , -10.858184 ,  -9.888906 ,  -9.361571 ,  -7.183362 ],
      dtype=float32)

In [32]:
# Store the rerank_scores in results_df

results_df['Reranked_scores'] = cross_rerank_scores

In [33]:
results_df

,IDs,Documents,Distances,Metadatas,Reranked_scores
0,7,"['GC 6003 Section A - Contract, Page 4', 'Sect...",0.26278409361839294,{'Policy_Name': 'Principal-Sample-Life-Insuran...,3.887320
1,10,"['GC 6004 Section B - Premiums, Page 3', 'Sect...",0.32783496379852295,{'Policy_Name': 'Principal-Sample-Life-Insuran...,2.811233
2,8,"['GC 6004 Section B - Premiums, Page 1', 'b . ...",0.3642001152038574,{'Policy_Name': 'Principal-Sample-Life-Insuran...,-4.489765
3,11,"['GC 6005 Section C - Policy Termination, Page...",0.38574838638305664,{'Policy_Name': 'Principal-Sample-Life-Insuran...,-4.911915
4,9,"['GC 6004 Section B - Premiums, Page 2', 'The ...",0.4074781835079193,{'Policy_Name': 'Principal-Sample-Life-Insuran...,-9.496572
5,23,"['GC 6007 Section B - Effective Dates, Page 7'...",0.4128356873989105,{'Policy_Name': 'Principal-Sample-Life-Insuran...,-9.795955
6,12,"['GC 6005 Section C - Policy Termination, Page...",0.41730251908302307,{'Policy_Name': 'Principal-Sample-Life-Insuran...,-10.858184
7,18,"['GC 6007 Section B - Effective Dates, Page 2'...",0.42553043365478516,{'Policy_Name': 'Principal-Sample-Life-Insuran...,-9.888906
8,71,"GC 6013 Section A - Member Life Insurance, Pag...",0.4263361692428589,{'Policy_Name': 'Principal-Sample-Life-Insuran...,-9.361571
9,32,['GC 6011 Section F - Individual Purchase Righ...,0.4362415075302124,{'Policy_Name': 'Principal-Sample-Life-Insuran...,-7.183362


In [34]:
# Return the top 3 results from semantic search

top_3_semantic = results_df.sort_values(by='Distances')
top_3_semantic[:3]

,IDs,Documents,Distances,Metadatas,Reranked_scores
0,7,"['GC 6003 Section A - Contract, Page 4', 'Sect...",0.26278409361839294,{'Policy_Name': 'Principal-Sample-Life-Insuran...,3.887320
1,10,"['GC 6004 Section B - Premiums, Page 3', 'Sect...",0.32783496379852295,{'Policy_Name': 'Principal-Sample-Life-Insuran...,2.811233
2,8,"['GC 6004 Section B - Premiums, Page 1', 'b . ...",0.3642001152038574,{'Policy_Name': 'Principal-Sample-Life-Insuran...,-4.489765


In [35]:
# Return the top 3 results after reranking

top_3_rerank = results_df.sort_values(by='Reranked_scores', ascending=False)
top_3_rerank[:3]

,IDs,Documents,Distances,Metadatas,Reranked_scores
0,7,"['GC 6003 Section A - Contract, Page 4', 'Sect...",0.26278409361839294,{'Policy_Name': 'Principal-Sample-Life-Insuran...,3.887320
1,10,"['GC 6004 Section B - Premiums, Page 3', 'Sect...",0.32783496379852295,{'Policy_Name': 'Principal-Sample-Life-Insuran...,2.811233
2,8,"['GC 6004 Section B - Premiums, Page 1', 'b . ...",0.3642001152038574,{'Policy_Name': 'Principal-Sample-Life-Insuran...,-4.489765


In [36]:
top_3_RAG = top_3_rerank[["Documents", "Metadatas"]][:3]

In [37]:
top_3_RAG

,Documents,Metadatas
0,"['GC 6003 Section A - Contract, Page 4', 'Sect...",{'Policy_Name': 'Principal-Sample-Life-Insuran...
1,"['GC 6004 Section B - Premiums, Page 3', 'Sect...",{'Policy_Name': 'Principal-Sample-Life-Insuran...
2,"['GC 6004 Section B - Premiums, Page 1', 'b . ...",{'Policy_Name': 'Principal-Sample-Life-Insuran...


In [38]:
# Define the function to generate the response. Provide a comprehensive prompt that passes the user query and the top 3 results to the model

def generate_response(query, results_df):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """
    messages = [
                {"role": "system", "content":  "You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents."},
                {"role": "user", "content": f"""You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents.
                                                You have a question asked by the user in '{query}' and you have some search results from a corpus of insurance documents in the dataframe '{top_3_RAG}'. These search results are essentially sections of the insurance document that is relevant to the user query.

                                                The column 'Documents' inside this dataframe contains the actual text from the policy document and the column 'Metadatas' contains the policy name and source section no.

                                                Use the Documents in '{top_3_RAG}' to answer the query '{query}'. Frame an informative answer from the 1st row in Documents column in {top_3_RAG} based on query and also, return the relevant policy names and section numbers as citations.

                                                Follow the guidelines below when performing the task.
                                                1. Provide relevant numbers or answers.
                                                2. Use all the information from the Documents column in {top_3_RAG} and then provide the shortlisted and concise answer based on {query}.
                                                3. Use the Metadatas columns in the dataframe to retrieve and cite the policy name(s) and section numbers(s) as citation.
                                                4. You are a customer facing assistant, so do not provide any information on internal workings, just answer the query directly and in a good and informative tone.

                                                The generated response should answer the query directly addressing the user and avoiding additional information. If you think that the query is not relevant to the document, reply that the query is irrelevant. Provide the final response as a well-formatted and easily readable text, it should have the answer to the query from the documents in {top_3_RAG} along with the citation. Provide your complete response first with all information, and then provide the citations.

                                                """},
              ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    return response['choices'][0]['message']['content'].split('\n')

In [39]:
def generate_response(query, results_df):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """
    # Combine relevant information from the results_df with the user's query.
    documents = top_3_RAG['Documents'][:2].tolist()  # Extract the content of the Documents column
    prompt = f"You have a question asked by the user in '{query}' and you have some search results from a corpus of insurance documents. Use the documents to answer the query '{query}'."

    # Prepare the messages input for the model.
    messages = [
        {"role": "system", "content": "You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents."},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": "\n".join(documents)}  # Include the documents in the conversation
    ]

    # Generate a response from the model.
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=150  # You can adjust the max_tokens to limit the response length
    )

    return response['choices'][0]['message']['content'].split('\n')


In [40]:
# Generate the response

response = generate_response(query, top_3_RAG)

In [41]:
response="\n".join(response)

In [42]:
response

'The premium for the insurance policy is due on the Date of Issue of the Group Policy and subsequently on the first of each Insurance Month. There is a Grace Period of 31 days allowed for the payment of each premium. During this Grace Period, the Group Policy will remain in force, but if the total premium due is not received by the insurance company before the end of the Grace Period, the policy will terminate. The Policyholder is responsible for making timely premium payments to ensure the continuous coverage under the policy.'